# 모델 성능 평가를 위한 데이터 셋 분리
## 데이터셋(Dataset)
- Train 데이터셋 (훈련 / 학습 데이터셋)
    - 모델을 학습시킬 때 사용할 데이터셋
- Validation 데이터셋 (검증 데이터셋)
    - 모델의 성능 중간 검증을 위한 데이터셋
- Test 데이터셋 (평가 데이터셋)
    - 모델의 성능을 최종적으로 측정하기 위한 데이터셋
    - `Test 데이터셋은 마지막에 모델의 성능을 측정하는 용도로 한번만 사용`

## Hold Out - Data분리 방식